In [11]:
import cv2
import numpy as np
import torch
from depth_anything_v2.dpt import DepthAnythingV2
import vtk
from PIL import Image


xFormers not available
xFormers not available


In [12]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 4070'

In [18]:

model = DepthAnythingV2(encoder='vitb', features=128, out_channels=[96, 192, 384, 768])
model.load_state_dict(torch.load('./checkpoints/depth_anything_v2_vitb.pth', map_location='cuda:0')) #cpu


# model = DepthAnythingV2(encoder='vits', features=64, out_channels=[48, 96, 192, 384])
# model.load_state_dict(torch.load('./checkpoints/depth_anything_v2_vits.pth', map_location='cuda:0'))

def pic23d(pic_path,save_path):
    model.cuda().eval()   #   cpu model.eval() 

    raw_img = cv2.imread(pic_path)  
    height, width = raw_img.shape[:2]  
    target_height = 600  
    scale_ratio = target_height / height  
    target_width = int(width * scale_ratio)  
    raw_img = cv2.resize(raw_img, (target_width, target_height))  
    
    depth = model.infer_image(raw_img) # HxW raw depth map
    depth_normalized = cv2.normalize(depth, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)  
    depth_display = (depth_normalized * 255).astype(np.uint8)  # 转换为8位图像以便显示  
    cv2.imwrite('./depth_map.png', depth_display)  

    fx = 50  # 焦距x  
    fy = 50# 焦距y  
    cx = raw_img.shape[1] / 2.0  # 光心x  
    cy = raw_img.shape[0] / 2.0  # 光心y  
    height, width = raw_img.shape[:2]  
    
    # 推断深度图  
    depth_ = depth_display/255
    # 深度图可能需要归一化或转换到实际的深度值，这里假设它已经是所需的格式  
    
    # 创建一个空的数组来存储空间位置坐标  
    pos = np.zeros((height * width, 3), dtype=np.float32)  
    
    # 遍历深度图的每个像素  


    # 创建一个空的数组来存储空间位置坐标  
    pos = np.zeros((height * width, 3), dtype=np.float32)  
    
    # 遍历深度图的每个像素  
    # Calculate indices for each pixel
    v_indices,u_indices = np.indices(depth_.shape)

    # Calculate spatial coordinates using vectorized operations
    X = (u_indices - cx) / fx
    Y = (v_indices - cy) / fy
    Z = depth_ * 10

    # Stack the coordinates into a single array
    pos = np.stack((X, Y, Z), axis=-1).reshape(-1, 3)
    
    with open("a.obj","w") as t:
        t.writelines("mtllib my_mtl.mtl"+"\n")
            
        for l in pos:
            L = "v "+str(l[0])+" "+str(l[1])+" "+str(l[2])+"\n"
            t.writelines(L )
        
        w_x = np.asarray([np.linspace(0.0, 1.0, num=width)]*height).reshape(width*height)
        h_y = np.asarray([np.linspace(1.0, 0.0, num=height)]*width).T.reshape(width*height)
        t_array_ = np.stack([w_x,h_y],axis=1)


        for i,j in t_array_:
            L = "vt " +str(i)+" "+str(j) +"\n"
            t.writelines(L )                
    #  
    #          
        t.writelines("\nusemtl my_mtl\n" )              
        b = np.array(range(height * width)).reshape([ height ,width])
        print(b.shape)  
                
        for i in range(1,b.shape[0]-1):
            for j in range(1,b.shape[1]-1):
                p = [ b[i,j] , b[i+1,j],b[i+1,j+1],b[i,j+1]]
        
                L = "f "+str(p[0])+"/"+ str(p[0])+" "+str(p[1])+"/"+ str(p[1])+" "+str(p[2])+"/"+ str(p[2])+" "+str(p[3])+"/"+ str(p[3])+"\n"
                t.writelines(L )
            


    with open("my_mtl.mtl","w") as m:  
        m.writelines(
    f"""
    newmtl my_mtl
    Ka 1 1 1
    Kd 1 1 1
    d 1
    Ns 0
    illum 1
    map_Kd {pic_path}
    """
        )




    def render_and_save_images(obj_file,  output_file, image_size=(800, 800)):
        # 读取模型
        reader = vtk.vtkOBJReader()
        reader.SetFileName(obj_file)
        reader.Update()

        # 创建一个映射器
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputConnection(reader.GetOutputPort())

        # 创建一个演员
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)

        # 读取贴图文件
        texture = vtk.vtkTexture()
        texture_reader = vtk.vtkJPEGReader()  # 假设贴图为JPEG格式
        texture_reader.SetFileName(pic_path)  # 根据.mtl文件中的路径设置
        texture_reader.Update()
        texture.SetInputConnection(texture_reader.GetOutputPort())
        
        # 将贴图应用到演员
        actor.SetTexture(texture)
        actor.SetOrientation(5,0,0)

        # 创建渲染窗口
        render_window = vtk.vtkRenderWindow()
        render_window.SetSize(image_size[0], image_size[1])  # 设置渲染窗口大小 

        # 创建左右视角的渲染器
        total_pic_count = 36
        renderers = []
        for i in range(total_pic_count):
            renderer = vtk.vtkRenderer()
            render_window.AddRenderer(renderer)
            renderers.append(renderer)
            # 禁用光照
            renderer.SetAmbient(100.0, 100.0, 100.0)  # 设置环境光
            actor.GetProperty().SetLighting(False)
            # 设置摄像机
            camera = vtk.vtkCamera()
            
            camera.SetFocalPoint(0, 0, 0)
            # if i == 0:  # 左视角
            #     camera.SetPosition(-2, 0, 30)  # 可以根据需要调整位置
            #     camera.SetViewUp(0, -1, 0)
            # else:  # 右视角
            #     camera.SetPosition(2, 0, 30)  # 右视角位置
            #     camera.SetViewUp(0, -1, 0)

      
            camera.SetPosition(-9+i*0.5, 0, 22)  # 可以根据需要调整位置
            camera.SetViewUp(0, -1, 0)

            renderer.SetActiveCamera(camera)

            # 添加演员到渲染器
            renderer.AddActor(actor)
            renderer.SetBackground(0.0, 0.0, 0.0)  # 背景颜色

            # 渲染
            render_window.Render()

            # 保存图像
            window_to_image_filter = vtk.vtkWindowToImageFilter()
            window_to_image_filter.SetInput(render_window)
            window_to_image_filter.ReadFrontBufferOff()  # 读取后台缓冲区
            window_to_image_filter.Update()

            writer = vtk.vtkPNGWriter()

            print("save_path:",save_path.replace("_/",f"{i+1}/"))
            writer.SetFileName(save_path.replace("_/",f"{i+1}/"))
        
            writer.SetInputConnection(window_to_image_filter.GetOutputPort())
            writer.Write()

    

    render_and_save_images('a.obj',  'output.png')


# pic_path = "./37.jpg"
# save_path = "./1.png"
# pic23d(pic_path,save_path)

In [21]:
import cv2
import os

def save_frames_from_video(video_path, output_folder, num_frames_range=[0,100]):
    if  ".mp4" in video_path:
        # 创建输出文件夹
        os.makedirs(output_folder, exist_ok=True)

        # 打开视频文件
        cap = cv2.VideoCapture(video_path)

        # 检查视频是否成功打开
        if not cap.isOpened():
            print("无法打开视频文件")
            return

        frame_count = 0


        for i in range(0,36):
            save_path = f"{output_folder}/{i+1}/"
            os.makedirs(save_path, exist_ok=True)

        #while (frame_count >= num_frames_range[0] and frame_count < num_frames_range[1]) :
        # for i in range(num_frames_range[0],num_frames_range[1]):
        pic_count = 0
        while True:
            ret, frame = cap.read()  # 读取帧
            if not ret:
                break  # 如果没有更多帧，退出循环
            frame_count += 1
            if  (frame_count >= num_frames_range[0] and frame_count < num_frames_range[1]) :
                pic_count+=1

                # 保存帧为图像文件
                frame_filename = os.path.join(output_folder, f'{pic_count}.jpg')
                cv2.imwrite(frame_filename, frame)
                

                save_path = frame_filename.replace("blender_gif/","blender_gif/_/")
                
                #print(">",frame_filename)
                pic23d(frame_filename,save_path)
                
    



        
        cap.release()  # 释放视频捕获对象
        print(f'已保存 {frame_count} 帧到 {output_folder}')
    else:
        for i in range(0,36):
            save_path = f"{output_folder}/{i+1}/"
            os.makedirs(save_path, exist_ok=True)

        pic23d(video_path,"./blender_gif/_/"+"1.jpg")




# 使用示例
# video_file = './video2.mp4'  # 视频文件路径

video_file = 'b3b.mp4'  # 视频文件路径
output_dir = './blender_gif/'    # 输出文件夹路径
save_frames_from_video(video_file, output_dir)

(600, 457)
save_path: ./blender_gif/1/1.jpg
save_path: ./blender_gif/2/1.jpg
save_path: ./blender_gif/3/1.jpg
save_path: ./blender_gif/4/1.jpg
save_path: ./blender_gif/5/1.jpg
save_path: ./blender_gif/6/1.jpg
save_path: ./blender_gif/7/1.jpg
save_path: ./blender_gif/8/1.jpg
save_path: ./blender_gif/9/1.jpg
save_path: ./blender_gif/10/1.jpg
save_path: ./blender_gif/11/1.jpg
save_path: ./blender_gif/12/1.jpg
save_path: ./blender_gif/13/1.jpg
save_path: ./blender_gif/14/1.jpg
save_path: ./blender_gif/15/1.jpg
save_path: ./blender_gif/16/1.jpg
save_path: ./blender_gif/17/1.jpg
save_path: ./blender_gif/18/1.jpg
save_path: ./blender_gif/19/1.jpg
save_path: ./blender_gif/20/1.jpg
save_path: ./blender_gif/21/1.jpg
save_path: ./blender_gif/22/1.jpg
save_path: ./blender_gif/23/1.jpg
save_path: ./blender_gif/24/1.jpg
save_path: ./blender_gif/25/1.jpg
save_path: ./blender_gif/26/1.jpg
save_path: ./blender_gif/27/1.jpg
save_path: ./blender_gif/28/1.jpg
save_path: ./blender_gif/29/1.jpg
save_path: .

In [ ]:
import requests
from PIL import Image
from io import BytesIO

def process_frame(pic_file_name, save_base="index.bin"):
    im = Image.open(pic_file_name)
    
    # Adjust the image size while maintaining the aspect ratio
    original_width, original_height = im.size
    target_height = 240
    target_width = int((target_height / original_height) * original_width)
    im = im.resize((target_width, target_height))  # Use LANCZOS filter for better quality

    # Optionally, convert to RGB if the image has an alpha channel
    if im.mode in ("RGBA", "LA") or (im.mode == "P" and "transparency" in im.info):
        im = im.convert("RGB")

    # Save as a binary file (using JPEG with adjustable quality)
    img_byte = BytesIO()
    im.save(img_byte, format='JPEG', quality=99)  # Adjust quality as needed
    img_data = img_byte.getvalue()

    # Write to binary file
    with open(save_base, "wb") as f:
        f.write(img_data)



In [ ]:
import os
pic_path = "C:/Users/Administrator/Pictures/blender_gif_3/"
# pic_path = "./blender_gif"
target_path = "6"
for i in range(0,180):
    for index, pic in enumerate(   sorted(os.listdir(pic_path+"/"+str(i)) ,key = lambda x: int(x.split(".")[0]))        ) :
        if ".jpg" in pic or ".png" in pic:
            # print(pic_path+pic)
            if not os.path.exists(f"F:/3d/{target_path}/"+str(i)+"/"):
                os.mkdir(f"F:/3d/{target_path}/"+str(i)+"/")
            process_frame(pic_path+"/"+str(i)+"/"+pic,f"F:/3d/{target_path}/"+str(i)+"/"+str(index)+".bin")

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: 'C:/Users/Administrator/Pictures/blender_gif_3//120'

In [3]:
import os
import cv2
import wave
import numpy as np
from pydub import AudioSegment

from PIL import Image
from io import BytesIO


def extract_audio(mp4_file, output_wav_file):
    """提取 MP4 文件中的音频并保存为 WAV 文件"""
    audio = AudioSegment.from_file(mp4_file, format="mp4")
    audio.export(output_wav_file, format="wav")
    print(f"Audio saved to {output_wav_file}")


def enhance_saturation(image, saturation_factor=1.5):
    """
    提升图像的色彩饱和度。
    :param image: OpenCV 图像 (BGR 格式)
    :param saturation_factor: 饱和度提升倍数
    :return: 提升饱和度后的图像
    """
    # 转换到 HSV 色彩空间
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # 提升饱和度
    h, s, v = cv2.split(hsv_image)
    s = np.clip(s * saturation_factor, 0, 255).astype(np.uint8)
    enhanced_hsv = cv2.merge([h, s, v])

    # 转换回 BGR
    enhanced_image = cv2.cvtColor(enhanced_hsv, cv2.COLOR_HSV2BGR)
    return enhanced_image


def process_frame(image, save_path):
    """处理单帧图像并保存为 .bin 文件"""
    # 调整图像大小，保持纵横比
    original_width, original_height = image.size
    if original_height>original_width:
        target_height = 240
        target_width = int((target_height / original_height) * original_width)
        image = image.resize((target_width, target_height), Image.LANCZOS)
    else:
        target_width = 240
        target_height = int((target_width / original_width) * original_height)
        image = image.resize(( target_width,target_height), Image.LANCZOS)       

    # 如果图像有 alpha 通道，转换为 RGB
    if image.mode in ("RGBA", "LA") or (image.mode == "P" and "transparency" in image.info):
        image = image.convert("RGB")

    # 保存为 JPEG 格式到内存
    img_byte = BytesIO()
    image.save(img_byte, format="JPEG", quality=80)
    img_data = img_byte.getvalue()

    # 写入二进制文件
    with open(save_path, "wb") as f:
        f.write(img_data)
    print(f"Frame saved to {save_path}")


def extract_frames(mp4_file, output_dir, saturation_factor=1.5):
    """提取 MP4 文件中的图像帧并保存为 .bin 文件，同时提升色彩饱和度"""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 使用 OpenCV 读取视频
    cap = cv2.VideoCapture(mp4_file)
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 提升色彩饱和度
        enhanced_frame = enhance_saturation(frame, saturation_factor)

        # 转换为 PIL 图像
        image = Image.fromarray(cv2.cvtColor(enhanced_frame, cv2.COLOR_BGR2RGB))

        # 保存为 .bin 文件
        bin_file = os.path.join(output_dir, f"frame_{frame_count:05d}.bin")
        process_frame(image, bin_file)
        frame_count += 1

    cap.release()
    print(f"Extracted {frame_count} frames to {output_dir}")


def convert_mp4_to_frames_and_audio(mp4_file, output_dir):
    """将 MP4 文件转换为图像帧和音频文件"""
    # 提取音频
    audio_file = os.path.join(output_dir, "audio.wav")
    extract_audio(mp4_file, audio_file)

    # 提取图像帧
    frames_dir = os.path.join(output_dir, "frames")
    extract_frames(mp4_file, frames_dir)


if __name__ == "__main__":
    mp4_file_path = "GIRL2.mp4"  # 输入 MP4 文件路径
    output_directory = "output"   # 输出目录

    convert_mp4_to_frames_and_audio(mp4_file_path, output_directory)

Audio saved to output\audio.wav
Frame saved to output\frames\frame_00000.bin
Frame saved to output\frames\frame_00001.bin
Frame saved to output\frames\frame_00002.bin
Frame saved to output\frames\frame_00003.bin
Frame saved to output\frames\frame_00004.bin
Frame saved to output\frames\frame_00005.bin
Frame saved to output\frames\frame_00006.bin
Frame saved to output\frames\frame_00007.bin
Frame saved to output\frames\frame_00008.bin
Frame saved to output\frames\frame_00009.bin
Frame saved to output\frames\frame_00010.bin
Frame saved to output\frames\frame_00011.bin
Frame saved to output\frames\frame_00012.bin
Frame saved to output\frames\frame_00013.bin
Frame saved to output\frames\frame_00014.bin
Frame saved to output\frames\frame_00015.bin
Frame saved to output\frames\frame_00016.bin
Frame saved to output\frames\frame_00017.bin
Frame saved to output\frames\frame_00018.bin
Frame saved to output\frames\frame_00019.bin
Frame saved to output\frames\frame_00020.bin
Frame saved to output\f

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15316\409322553.py:49: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  image = image.resize(( target_width,target_height), Image.LANCZOS)


Frame saved to output\frames\frame_00033.bin
Frame saved to output\frames\frame_00034.bin
Frame saved to output\frames\frame_00035.bin
Frame saved to output\frames\frame_00036.bin
Frame saved to output\frames\frame_00037.bin
Frame saved to output\frames\frame_00038.bin
Frame saved to output\frames\frame_00039.bin
Frame saved to output\frames\frame_00040.bin
Frame saved to output\frames\frame_00041.bin
Frame saved to output\frames\frame_00042.bin
Frame saved to output\frames\frame_00043.bin
Frame saved to output\frames\frame_00044.bin
Frame saved to output\frames\frame_00045.bin
Frame saved to output\frames\frame_00046.bin
Frame saved to output\frames\frame_00047.bin
Frame saved to output\frames\frame_00048.bin
Frame saved to output\frames\frame_00049.bin
Frame saved to output\frames\frame_00050.bin
Frame saved to output\frames\frame_00051.bin
Frame saved to output\frames\frame_00052.bin
Frame saved to output\frames\frame_00053.bin
Frame saved to output\frames\frame_00054.bin
Frame save

In [13]:
from perspective2d import PerspectiveFields as _PerspectiveFields
import torch
import math
import cv2
from perspective2d import PerspectiveFields as _PerspectiveFields


class PerspectiveFields(torch.nn.Module):
    """
    Thin wrapper around perspective fields model for inferring camera intrinsics
    """
    VERSIONS = {
        "Paramnet-360Cities-edina-centered",
        "Paramnet-360Cities-edina-uncentered",
    }

    def __init__(
            self,
            version="Paramnet-360Cities-edina-uncentered",
            #
            #Paramnet-360Cities-edina-centered
            device="cuda",
    ):
        """
        Args:
            version (str): underlying model version to use. Valid options are:
                {"Paramnet-360Cities-edina-uncentered", "Paramnet-360Cities-edina-centered"}
            device (str): device to store tensors on. Default is "cuda"
        """
        # Call super first
        super().__init__()

        # Sanity check version
        assert version in self.VERSIONS,\
            f"Got invalid PerspectiveFields version! Valid options: {self.VERSIONS}, got: {version}"

        # Load model
        self.device = device
        self.model = _PerspectiveFields(version)
        self.model.to(self.device)
        self.model.eval()

    def estimate_camera_intrinsics(self, input_path):
        """
        Estimates the K camera intrinsics matrix using PerspectiveFields model

        Args:
            input_path (str): Absolute path to the image from which to infer camera intrinsics

        Returns:
            np.ndarray: Estimated intrinsics matrix
        """
        # Load the image
        img_bgr = cv2.imread(input_path)

        # Run predictions
        predictions = self.model.inference(img_bgr=img_bgr)

        # Compute intrinsics
        height, width, _ = img_bgr.shape
        # Convert vfov to radians
        vfov_rad = math.radians(predictions['pred_general_vfov'].item())
        # Compute focal length in pixels
        f = (height / 2) / math.tan(vfov_rad / 2)
        # Compute fx and fy
        fx = f * predictions['pred_rel_focal'].item()
        fy = f
        # Compute cx and cy
        cx = width / 2.0
        cy = height / 2.0
        # Construct the camera intrinsics matrix
        K = torch.tensor([
            [fx, 0, cx],
            [0, fy, cy],
            [0, 0, 1]
        ])
        print(K)

        return K
    
import os
import numpy as np
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import torch

# import digital_cousins
from metric_depth.depth_anything_v2.dpt import DepthAnythingV2 as _DepthAnythingV2

# from digital_cousins.utils.processing_utils import process_depth_linear, unprocess_depth_linear

def rescale_image(img, in_limits, out_limits):
    """
    Rescales image from having values in range @in_limits to range @out_limits

    Args:
        img (np.ndarray): Image to be rescaled
        in_limits (2-tuple): (min, max) range of the input image
        out_limits (2-tuple): (min, max) range of the rescaled image

    Returns:
        np.ndarray: Rescaled image
    """
    # Out shape if specified should be (H, W) 2-tuple
    # Keep absolute range to be from 0 to 10 --> renormalize to 0 - 1
    in_min, in_max = in_limits
    out_min, out_max = out_limits
    scale_factor = abs(out_max - out_min) / abs(in_max - in_min)
    out_tf = (out_max + out_min) / 2.0
    in_tf = (in_max + in_min) / 2.0
    img = (img - in_tf) * scale_factor + out_tf
    return img



def process_depth_linear(depth, in_limits=(0, 10), out_limits=(0, 1), out_shape=None, use_16bit=True):
    """
    Discretizes a linear depth map from range @in_limits to range @out_limits, and optionally resizes it to @out_shape

    Args:
        depth (np.ndarray): Input depth map with metric values
        in_limits (2-tuple): (min, max) range of the input image
        out_limits (2-tuple): (min, max) range of the processed image
        out_shape (None or 2-tuple): If specified, the (H, W) @depth should be resized to
        use_16bit (bool): Whether to use 16-bit or 8-bit uint encoding when rescaling the image

    Returns:
        np.ndarray: Processed depth image.
    """
    # Out shape if specified should be (H, W) 2-tuple
    # Keep absolute range to be from 0 to 10 --> renormalize to 0 - 1
    in_min, in_max = in_limits
    foreground = np.where(depth <= in_max)
    background = np.where(depth > in_max)
    depth = rescale_image(img=depth, in_limits=in_limits, out_limits=out_limits)

    # Zero out background
    bit_size = 16 if use_16bit else 8
    dtype = np.uint16 if use_16bit else np.uint8
    depth[background] = 0.0
    # Multiply by 2 ** 16
    depth = depth * (2 ** bit_size)
    depth = depth.astype(dtype)
    if out_shape is not None:
        depth = cv2.resize(depth, (out_shape[1], out_shape[0]))
    return depth


def unprocess_depth_linear(depth, in_limits=(0.0, 1.0), out_limits=(0.0, 10.0)):
    """
    Unnormalizes a linear depth map from range @in_limits to range @out_limits. This process is the inverse
    of @procedss_depth_linear)

    Args:
        depth (np.ndarray): Input depth map with normalized values (the output of @process_depth_linear)
        in_limits (2-tuple): (min, max) range of the input image
        out_limits (2-tuple): (min, max) range of the unprocessed image
    
    Returns:
        np.ndarray: Unnormalized depth image.
    """
    # Map to float, divide by 2**16, then invert transform
    depth = depth.astype(float)
    depth = depth / (2 ** 16)

    # Keep unnormalize from 0 - 1 --> 0 - 10
    in_min, in_max = in_limits
    out_min, out_max = out_limits
    scale_factor = abs(out_max - out_min) / abs(in_max - in_min)
    out_tf = (out_max + out_min) / 2.0
    in_tf = (in_max + in_min) / 2.0
    depth = (depth - in_tf) * scale_factor + out_tf
    return depth





class DepthAnythingV2(torch.nn.Module):
    """
    Thin wrapper around DepthAnything V2 model for inferring depth maps
    """
    # Maps backbone size to configuration to use
    CONFIGS = {
        'small': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
        'base': {'encoder': 'vitb', 'features': 128, 'out_channels': [96, 192, 384, 768]},
        'large': {'encoder': 'vitl', 'features': 256, 'out_channels': [256, 512, 1024, 1024]},
    }

    # Maps dataset name to max depth to use
    DATASET_MAX_DEPTHS = {
        "hypersim": 20,
        "vkitti": 80,
    }

    def __init__(
            self,
            backbone_size="base",
            backbone_dataset="hypersim",
            max_depth=None,
            device="cuda",
    ):
        """
        Args:
            backbone_size (str): Size of underlying ViT model to use. Options are {'small', 'base', 'large'}
            backbone_dataset (str): Which training dataset the underlying model was trained on.
                Use "hypersim" for indoor estimation, "vkitti" for outdoor model
            max_depth (None or int): Maximum depth (m) to use for depth estimation. If None, will use a default
                based on @backbone_dataset
            device (str): device to store tensors on. Default is "cuda"
        """
        # Call super first
        super().__init__()

        # Sanity check values
        assert backbone_size in self.CONFIGS,\
            f"Got invalid DepthAnythingV2 backbone_size! Valid options: {self.CONFIGS.keys()}, got: {backbone_size}"
        assert backbone_dataset in self.DATASET_MAX_DEPTHS,\
            f"Got invalid DepthAnythingV2 backbone_dataset! Valid options: {self.DATASET_MAX_DEPTHS.keys()}, got: {backbone_dataset}"

        self.max_depth = self.DATASET_MAX_DEPTHS[backbone_dataset] if max_depth is None else max_depth
        self.device = 'cuda:0'

        #Load model
        self.model = _DepthAnythingV2(**{**self.CONFIGS[backbone_size], "max_depth": self.max_depth})
        self.model.load_state_dict(
            torch.load(
                './checkpoints/depth_anything_v2_metric_hypersim_vitb.pth',
                map_location=self.device,
                weights_only=True,
            ),
        )
        self.model.to(self.device).eval()

        # self. model = _DepthAnythingV2(encoder='vitb', features=128, out_channels=[96, 192, 384, 768])
        # self.model.load_state_dict(torch.load('./checkpoints/depth_anything_v2_vitb.pth', map_location='cuda:0')) #cpu
        # self.model.cuda().eval() 
        #depth = model.infer_image(raw_img) # HxW raw depth map




    def estimate_depth_linear(self, input_path, output_path, depth_limits=(0, 10.0)):
        """
        Estimates linear depth map using DepthAnythingV2 model.

        NOTE: depth linear implies that the outputted map is taken with respect to the image plane,
            NOT the camera position!

        Args:
            input_path (str): Absolute path to the image from which to infer linear depth map
            output_path (str): Absolute path to the location for the estimated linear depth map image
            depth_limits (2-tuple): (min, max) values from the depth map to normalize scaling when saving the
                depth map image

        Returns:
            np.ndarray: Estimated linear depth map
        """
        img = cv2.imread(input_path)
        H, W, _ = img.shape
        pred = self.model.infer_image(img)  # HxW depth map in meters in numpy

        # Save output; normalize and convert the depth image to an 8-bit format if necessary
        Path(os.path.dirname(output_path)).mkdir(parents=True, exist_ok=True)
        depth = process_depth_linear(depth=pred, in_limits=depth_limits)
        Image.fromarray(depth).save(output_path)

        return depth

    def crop_center(self, image, crop_percent=1):
        """
        Crops the center of the image based on the specified crop percentage.
        Works for both single-channel (grayscale/depth) and three-channel (RGB) images.

        Params:
            image (np.ndarray): A numpy array representing the image. Can be 2D (single-channel) or 3D (multi-channel).
            crop_percent (int): The percentage of the image to retain in the center. Default is 0.8.

        Returns:
        - np.ndarray: A numpy array representing the cropped image.
        """

        if crop_percent <= 0 or crop_percent > 1:
            raise ValueError("crop_percent must be between 0 and 1")

        # Check the number of dimensions to handle both single and multi-channel images
        if image.ndim == 3:  # Multi-channel image
            height, width, _ = image.shape
        elif image.ndim == 2:  # Single-channel image
            height, width = image.shape
        else:
            raise ValueError("Unsupported image shape")

        # Calculate the new dimensions
        new_height = int(height * crop_percent)
        new_width = int(width * crop_percent)

        # Calculate margins
        top_margin = (height - new_height) // 2
        left_margin = (width - new_width) // 2

        # Crop the image based on the number of dimensions
        if image.ndim == 3:
            cropped_image = image[top_margin:top_margin + new_height, left_margin:left_margin + new_width, :]
        else:  # image.ndim == 2
            cropped_image = image[top_margin:top_margin + new_height, left_margin:left_margin + new_width]

        print(f"cropped_img_shape: {cropped_image.shape}")
        return cropped_image

ModuleNotFoundError: No module named 'perspective2d'

In [12]:
depth_max_limit = 20

def compute_point_cloud_from_depth(depth, K, cam_to_img_tf=None, world_to_cam_tf=None, visualize_every=0,
                                   grid_limits=None):
    """
    Computes point cloud from depth image.

    Args:
        depth (np.ndarray): Input depth map with normalized values (the output of @process_depth_linear)
        K (np.ndarray): 3x3 cam intrinsics matrix
        cam_to_img_tf (np.ndarray): 4x4 Camera to image coordinate transformation matrix.
                    omni cam_to_img_tf is T.pose2mat(([0, 0, 0], T.euler2quat([np.pi, 0, 0])))
        world_to_cam_tf (np.ndarray): 4x4 World to camera coordinate transformation matrix.
        visualize_every (int): Step size when uniformly sampling points in the resulting point cloud to visualize.
        grid_limits (float): Visualization plot grid limits.
    
    Returns:
        np.ndarray: Resulting point cloud.
    """
    
    h, w = depth.shape
    y, x = np.meshgrid(np.arange(h), np.arange(w), indexing="ij", sparse=False)
    assert depth.min() >= 0
    u = x
    v = y
    uv = np.dstack((u, v, np.ones_like(u)))

    Kinv = np.linalg.inv(K)

    pc = depth.reshape(-1, 1) * (uv.reshape(-1, 3) @ Kinv.T)
    pc = pc.reshape(h, w, 3)

    # import open3d as o3d
    # pcd = o3d.geometry.PointCloud()
    # pcd.points = o3d.utility.Vector3dVector(pc.reshape(-1, 3))
    # # pcd.colors = o3d.utility.Vector3dVector(rgb.reshape(-1, 3) / 255.0)
    # o3d.visualization.draw_geometries([pcd])

    # If no tfs, use identity matrix
    cam_to_img_tf = np.eye(4) if cam_to_img_tf is None else cam_to_img_tf
    world_to_cam_tf = np.eye(4) if world_to_cam_tf is None else world_to_cam_tf

    pc = np.concatenate([pc.reshape(-1, 3), np.ones((h * w, 1))], axis=-1)  # shape (H*W, 4)

    # Convert using camera transform
    # Create (H * W, 4) vector from pc
    pc = (pc @ cam_to_img_tf.T @ world_to_cam_tf.T)[:, :3].reshape(h, w, 3)
    return pc,h,w
device = "cuda:0"
input_path = "./Q8.jpg"
save_dir = "."




raw_img = cv2.imread(input_path)  
height, width = raw_img.shape[:2]  
target_height = 600  
scale_ratio = target_height / height  
target_width = int(width * scale_ratio)  
raw_img = cv2.resize(raw_img, (target_width, target_height))  
cv2.imwrite(input_path, raw_img)

camera_intrinsics_matrix = None
if camera_intrinsics_matrix is None:
   
    intrinsics_estimator = PerspectiveFields(device=device)
    intrinsics = intrinsics_estimator.estimate_camera_intrinsics(input_path=input_path)
    intrinsics_estimator.to('cpu')
    del intrinsics_estimator
    camera_intrinsics_matrix = np.array(intrinsics)


depth_path = f"{save_dir}/step_1_depth.png"
depth_limits = np.array([0, depth_max_limit])
depth_estimator = DepthAnythingV2(device=device)



depth_estimator.estimate_depth_linear(input_path=input_path, output_path=depth_path, depth_limits=depth_limits)
depth_estimator.to('cpu')
del depth_estimator

depth = unprocess_depth_linear(np.array(Image.open(depth_path)), out_limits=depth_limits)
pc = compute_point_cloud_from_depth(depth=depth, K=camera_intrinsics_matrix)

NameError: name 'PerspectiveFields' is not defined

In [11]:
import open3d as o3d
visualize = True
if visualize:
    # Explicitly manage visualization process to prevent conflict with og launching process
    def vis():
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(pc[0].reshape(-1, 3))
        # pcd.colors = o3d.utility.Vector3dVector(rgb.reshape(-1, 3) / 255.0)
        o3d.visualization.draw_geometries([pcd])
    # vis_process = multiprocessing.Process(target=vis)
    # vis_process.start()
    # vis_process.join()
vis()

In [8]:
for l in pc[0]:
    print( "v "+str(l[0][0])+" "+str(l[0][1])+" "+str(l[0][2])+"\n")
    break
    

v -5.718049773986422 -8.629110509991733 19.99969482421875



In [22]:
pos ,height ,width= pc  #pc,h,w
pos = pos.reshape(-1, 3)
print(pos.shape)
with open("a.obj","w") as t:
    t.writelines("mtllib my_mtl.mtl"+"\n")
        
    for l in pos:
        L =  "v "+str(l[0])+" "+str(l[1])+" "+str(l[2])+"\n"
        t.writelines(L )
    
    w_x = np.asarray([np.linspace(0.0, 1.0, num=width)]*height).reshape(width*height)
    h_y = np.asarray([np.linspace(1.0, 0.0, num=height)]*width).T.reshape(width*height)
    t_array_ = np.stack([w_x,h_y],axis=1)


    for i,j in t_array_:
        L = "vt " +str(i)+" "+str(j) +"\n"
        t.writelines(L )                
#  
#          
    t.writelines("\nusemtl my_mtl\n" )              
    b = np.array(range(height * width)).reshape([ height ,width])
    print(b.shape)  
            
    for i in range(1,b.shape[0]-1):
        for j in range(1,b.shape[1]-1):
            p = [ b[i,j] , b[i+1,j],b[i+1,j+1],b[i,j+1]]
    
            L = "f "+str(p[0])+"/"+ str(p[0])+" "+str(p[1])+"/"+ str(p[1])+" "+str(p[2])+"/"+ str(p[2])+" "+str(p[3])+"/"+ str(p[3])+"\n"
            t.writelines(L )
        


with open("my_mtl.mtl","w") as m:  
    m.writelines(
f"""
newmtl my_mtl
Ka 1 1 1
Kd 1 1 1
d 1
Ns 0
illum 1
map_Kd {input_path}
"""
    )


(270000, 3)
(600, 450)


In [1]:
import digital_cousins
# If you store the offline dataset elsewhere, please uncomment the following line and put the directory here
# digital_cousins.ASSET_DIR = "~/assets"

import os
from PIL import Image
import numpy as np
import argparse

# import digital_cousins

TEST_DIR = os.path.dirname("test_img.png")
SAVE_DIR = f"{TEST_DIR}/test_acdc_output"
TEST_IMG_PATH = f"{TEST_DIR}/test_img.png"
CAPTION = "Fridge. Cabinet."


def test_perspective_fields(TEST_IMG_PATH):
    from digital_cousins.models.perspective_fields import PerspectiveFields
    PerspectiveFields().estimate_camera_intrinsics(input_path=TEST_IMG_PATH)


def test_depth_anything_2(TEST_IMG_PATH):
    from digital_cousins.models.depth_anything_v2 import DepthAnythingV2
    depth_path = f"test_depth.png"
    DepthAnythingV2().estimate_depth_linear(input_path=TEST_IMG_PATH, output_path=depth_path)





def main():
    # Run all tests
    print()
    print("*" * 30)
    print("Starting tests...")
    print("*" * 30)
    print()
    TEST_IMG_PATH = "./test_img.png"
  
    test_perspective_fields(TEST_IMG_PATH)
    test_depth_anything_2(TEST_IMG_PATH)
 
main()

    # Final test -- OG should always come at the end
    # This og test cannot run together with test_acdc_step_3
    # because the simulator can only be launched once, and after calling og.shutdown(), the whole process will terminate
    # test_og(args)


# if __name__ == "__main__":
#     # Define args
#     parser = argparse.ArgumentParser()
  


******************************
Starting tests...
******************************



c:\Users\vv\Miniconda3\envs\llama2\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
xFormers not available
xFormers not available


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
.//checkpoints
['depth_anything_v2_metric_hypersim_vitb.pth', 'depth_anything_v2_vitb.pth', 'depth_anything_v2_vits.pth']


In [ ]:
import bpy
import os

# 设置保存路径



# 获取场景中的Cube对象
cube = bpy.data.objects.get("圆环")
if not cube:
    raise ValueError("找不到名为 'Cube' 的对象，请确保它存在")

# 配置渲染设置
bpy.context.scene.render.image_settings.file_format = 'PNG'  # 渲染为PNG格式
bpy.context.scene.render.resolution_x = 800  # 宽度
bpy.context.scene.render.resolution_y = 800  # 高度

# 旋转和渲染逻辑
for index, i in enumerate( list( range(-180, 180, 5))):
    
    save_path = f"C:/Users/Administrator/Pictures/blender_gif/{index}/"
    os.makedirs(save_path, exist_ok=True)
    
    # 旋转Cube对象
    cube.rotation_euler[2] = i * (3.14159 / 180)  # 将角度转换为弧度
    
    for frame in range(1,43):
    
        # 设置帧号（用于保存文件名）
        bpy.context.scene.frame_set(frame)  # 每5度为一帧
    
        # 设置输出文件路径
        file_name = f"{frame}.png"
        bpy.context.scene.render.filepath = os.path.join(save_path, file_name)
    
        # 渲染当前帧
        #bpy.ops.render.render(write_still=True)
        bpy.ops.render.opengl(write_still=True, view_context=True)
    index+=1
  

print("渲染完成！所有图片保存在：", save_path)